### Importing imports

In [1]:
#import tensorflow library
import tensorflow as tf

import matplotlib.pyplot as plt

#imports ImageDataGenerator class from Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


### Preprocessing and data augmentation

In [2]:

#creating ImageDataGenerator object
train_datagen = ImageDataGenerator(
    #rescales pixel values to 0-1 range from 0-255 range
    rescale = 1. /255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    #shearing the image for variations 
    shear_range = 0.2,
    #zoom transformation for variations
    zoom_range = 0.2,
    #randomly flips the images horizontally, helps prevent overfitting
    #also helps doubles the size of training dataset, providing more examples to learn from
    horizontal_flip = True,
    #20% of the training data will be used as validation data
    validation_split = 0.2,
    fill_mode = 'nearest'
)

### generate batches of data for model training, drawn directly from a designated directory

In [3]:

#Creates a data generator that loads images from the specified directory
train_generator = train_datagen.flow_from_directory(
    r'archive\asl_alphabet_train\asl_alphabet_train',
    # resizes images to 64x64 pixels
    target_size = (128, 128),
    #specifies the batch size for training
    batch_size = 32,
    #indicates categorical classification
    class_mode = 'categorical',
    #Loads images in grayscale mode
    color_mode = 'grayscale',
    subset = 'training',
)


validation_generator = train_datagen.flow_from_directory(
    r'archive\asl_alphabet_train\asl_alphabet_train',
    target_size = (128, 128),
    batch_size = 32,
    class_mode = 'categorical',
    color_mode = 'grayscale',
    subset = 'validation',
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


### Model Creation


In [4]:

#defining CNN model using tensorflow and keras
#creates a sequential model where layers are linearly stacked
model = tf.keras.models.Sequential([
    #takes 64x64 pixel grayscale images as input
    tf.keras.layers.Input(shape = (128, 128, 1)),
    #adds convolutional layers to extract features
    #32 filter(kernals) in the layer
    # size of the filter is 3x3
    # uses relu activation function
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (64, 64, 1)),
    #adds pooling layer to reduce dimensionality 
    #size of each filter is 2x2
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    # this layer converts the multi-dimensional data into a single, long vector
    tf.keras.layers.Flatten(),

    #dense layer where each neuron is connected to every neuron in the previous layer
    #first one has 128 neurons, second one has 29 neurons
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(29, activation='softmax')
])

c:\Users\tonyj\Desktop\2024 Fall\CPS 843\ASL\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Early stopping and model fit

In [ ]:

#compiles the model
#uses adam optimizer
#loss function is categorical crossentropy (measures the difference between model's predictions and true label)
#specifies the metric used to evaluate the mode's performance during training
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

#early stopping technique to prevent overfitting
early_stopping = EarlyStopping(
    monitor = "val_loss",
    #if validation loss does not improve for 3 epochs, training will stop
    patience = 3,
    #ensure model's weights are restored from the epoch with the best validation loss
    restore_best_weights = True,
)

#initializes training of the model (actual learning happens)
#train generator is the data generator created earlier
#model goes through the training data 20 times
history = model.fit(
    train_generator,
    epochs=15,
    validation_data = validation_generator,
    callbacks=[early_stopping],
)

model.save('asl_model.h5')

c:\Users\tonyj\Desktop\2024 Fall\CPS 843\ASL\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step - accuracy: 0.1486 - loss: 2.9504

c:\Users\tonyj\Desktop\2024 Fall\CPS 843\ASL\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 883s 405ms/step - accuracy: 0.1487 - loss: 2.9502 - val_accuracy: 0.2753 - val_loss: 2.4238
Epoch 2/15
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 284s 131ms/step - accuracy: 0.5062 - loss: 1.5832 - val_accuracy: 0.3780 - val_loss: 2.0707
Epoch 3/15
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 267s 123ms/step - accuracy: 0.6446 - loss: 1.1007 - val_accuracy: 0.4141 - val_loss: 2.0521
Epoch 4/15
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 267s 123ms/step - accuracy: 0.7151 - loss: 0.8688 - val_accuracy: 0.4657 - val_loss: 1.7527
Epoch 5/15
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 305s 140ms/step - accuracy: 0.7627 - loss: 0.7221 - val_accuracy: 0.5295 - val_loss: 1.5887
Epoch 6/15
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 268s 123ms/step - accuracy: 0.7929 - loss: 0.6315 - val_accuracy: 0.5503 - val_loss: 1.5106
Epoch 7/15
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 268s 123ms/step - accuracy: 0.8153 - loss: 0.5579 - val_accuracy: 0.5574 - val_loss: 1.5117
Epoch 8/15
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 269s 124ms/step - accuracy: 0.8

### Plotting the graph

In [7]:

# plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel("Accuracy")
plt.ylabel('Epoch')
plt.legend(['Train', 'Validation'], loc = 'upper left')
plt.show()

#plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc = 'upper left')
plt.show()

NameError: name 'history' is not defined